In [2]:
!pip install pyspark
!pip install pyngrok


In [3]:
from pyspark.sql import SparkSession
import string
from pyngrok import ngrok
import re

spark = SparkSession.builder \
    .appName("Word Count") \
    .getOrCreate()


stop_words = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves",
    "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are",
    "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about",
    "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up",
    "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when",
    "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no",
    "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should",
    "now", "d", "ll", "m", "o", "re", "ve", "y", "ain", "aren", "couldn", "didn", "doesn", "hadn", "hasn", "haven",
    "isn", "ma", "mightn", "mustn", "needn", "shan", "shouldn", "wasn", "weren", "won", "wouldn"
])

def clean_word(word):
    return re.sub(r'[^\w\s]', '', word).lower()

files = ['/content/input1.txt', '/content/input2.txt']
rdd = spark.sparkContext.textFile(files[1]).union(spark.sparkContext.textFile(files[0]))

uncleaned_word_counts = (
    rdd.flatMap(lambda line: line.split())
       .map(lambda word: (word, 1))
       .reduceByKey(lambda a, b: a + b)
       .sortBy(lambda x: x[1], ascending=False)
)
uncleaned_output = uncleaned_word_counts.collect()
with open('output_1.txt', 'w') as f:
    for word, count in uncleaned_output:
        f.write(f"{word}: {count}\n")

word_counts = (
    rdd.flatMap(lambda line: line.split())
       .map(clean_word)
       .filter(lambda word: word not in stop_words)
       .map(lambda word: (word, 1))
       .reduceByKey(lambda a, b: a + b)
       .sortBy(lambda x: x[1], ascending=False)
)

output = word_counts.collect()
with open('output_3.txt', 'w') as f:
    for word, count in output:
        f.write(f"{word}: {count}\n")

print("Top words:")
for word, count in output[:25]:
    print(f"{word}: {count}")

ngrok.set_auth_token("2oY32y1o2wSGEJe5t6a6WZAnWxW_7cMg48xpKAosLShJ5LDdX")
public_url = ngrok.connect(4040)
print(public_url)

Top words:
mr: 889
said: 638
elizabeth: 603
could: 593
would: 532
one: 425
mrs: 387
darcy: 378
much: 373
must: 364
miss: 353
know: 339
bennet: 307
little: 293
never: 292
think: 286
time: 281
well: 281
jane: 271
though: 267
bingley: 261
man: 253
see: 240
may: 237
might: 232


NgrokTunnel: "https://7778-34-105-21-235.ngrok-free.app" -> "http://localhost:4040"
